Spatial Data Science

Hoemwork 9

Mariam Hovhannisyan

In [1]:
%matplotlib inline

from pysal.model import spreg
from pysal.lib import weights
from pysal.explore import esda
from shapely.geometry import Point
from tqdm import tqdm
from shapely.ops import nearest_points
from pyproj import CRS
from scipy import stats
import statsmodels.formula.api as sm
import numpy as np
import osmnx as ox, geopandas as gpd, pandas as pd, networkx as nx
import matplotlib.pyplot as plt
import seaborn

Compute and plot the distances from all restaurants in Yerevan to your house.

In [2]:
# download and project a street network
G = ox.graph_from_place('Yerevan, Armenia', network_type="walk")
G_proj = ox.project_graph(G)
#fig, ax = ox.plot_graph(G_proj, figsize=(20,20), node_size=2, node_color='#999999', node_edgecolor='none', node_zorder=2,
 #                       edge_color='#555555', edge_linewidth=0.7, edge_alpha=1)

In [ ]:
tags = {"amenity": "restaurant"}
rests = ox.geometries_from_place('Yerevan, Armenia', tags)
rests['index'] = range(0,407)
rests.set_index(rests['index'], inplace = True, drop = True)

rests = rests[(rests['geometry'].geom_type == "Point")]
rests = rests[['geometry']]
rests

In [ ]:
from tqdm import tqdm
from shapely.ops import nearest_points

dict_dist_to_home = {}
home = Point(40.171427, 44.511065)

for i in tqdm(range(len(rests))):
    point = rests.loc[i, "geometry"]
    origin = ox.distance.nearest_nodes(G_proj, point.coords[0][0], point.coords[0][1])
    candidate_pt, p2 = nearest_points(home, point)
    destination = ox.distance.nearest_nodes(G_proj, candidate_pt.coords[0][0], candidate_pt.coords[0][1])
    try:
        if origin!=destination:
            dist = nx.shortest_path_length(G_proj, origin, destination, weight = "length")
    except:
        
        dist = 'NaN'
        pass

    print("Distance from listing {} to my home: ".format(i), dist)
    dict_dist_to_home[i] = dist

In [ ]:
rest_dists = np.fromiter(dist_to_rest.values(), dtype=float)
plt.hist(rest_dists, bins=50, alpha=0.6)
plt.show()

In [ ]:
rests['dist_from_home'] = rests.index.map(dist_to_rest)
rests = rests[rests['dist_from_home']!='NaN']
rests.reset_index(inplace=True, drop=True)
print(len(rests))
rests.head()

In [ ]:
fig,ax = plt.subplots(1,figsize=(24, 30))

# ox.plot_graph(G_proj, ax=ax, show=False, close=False, \
#               node_color='grey', node_size=0, edge_linewidth=1.5, node_alpha=0.8, node_zorder=1, edge_color='grey', \
#               edge_alpha=0.5)

rests.plot(ax=ax, column = 'dist_from_home', cmap='coolwarm', s=15, legend=False, zorder=3)
home.plot(ax=ax, color='green', alpha=0.8)

cbax = fig.add_axes([0.8, 0.125, 0.02, 0.755])
sm = plt.cm.ScalarMappable(cmap='coolwarm', \
                          norm = plt.Normalize(vmin=min(rests.dist_from_home), vmax=max(rests.dist_from_home)))

sm._A = []

# draw colormap into cbax

fig.colorbar(sm, cax=cbax, format="%d")

ctx.add_basemap(
    ax, 
    crs=rests.crs.to_string(), 
    source=ctx.providers.Stamen.Toner
)